# ***LLaMA-3.2-3B (or other models) Eval on SiQA***

In [ ]:
import os
os.environ["HF_TOKEN"] = ###

In [ ]:
import os
os.environ["WANDB_API_KEY"] = ###

In [ ]:
!pip install wandb

**Mount Google Drive first!!!**
**Then change the following path to the folder which has the needed dataset/files**

In [3]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 149.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install torch
!pip install --upgrade jax
!pip install jaxlib

In [ ]:
import transformers
print(transformers.__version__)

4.48.3


# **Dataset Prep**

**Note: The SiQA dataset comes with train and dev files and the labels are in separate files. These paths are where these files are located.**

In [14]:
train_file = "./socialiqa-train-dev/train.jsonl"
train_labels = "./socialiqa-train-dev/train-labels.lst"

test_file = "./socialiqa-train-dev/dev.jsonl"
test_labels = "./socialiqa-train-dev/dev-labels.lst"

In [15]:
import json

def load_data(text, labels):
    # Load the JSONL file
    with open(text, 'r') as f:
      data = [json.loads(line) for line in f]

    # Load the labels file
    with open(labels, 'r') as f:
      answers = [line.strip() for line in f]

    return data, answers

In [ ]:
data, labels = load_data(train_file, train_labels)

In [18]:
# clean up data and make prompt
from datasets import Dataset

def clean_data(data, labels):
  label_to_key = {"1": "answerA", "2": "answerB", "3": "answerC"}
  key_to_choice = {"answerA": "A", "answerB": "B", "answerC": "C"}

  processed_data = []
  for entry, label in zip(data, labels):
    # Retrieve the correct answer text based on the label
    correct_answer = entry[label_to_key[label]]
    correct_choice = key_to_choice[label_to_key[label]]

    # Construct the prompt
    prompt = (
        f"Context: {entry['context']}\n"
        f"Question: {entry['question']}\n"
        "Options:\n"
        f"  A: {entry['answerA']}\n"
        f"  B: {entry['answerB']}\n"
        f"  C: {entry['answerC']}\n"
        "The correct answer is:\n"
    )

    processed_data.append({
        "prompt": prompt,
        "choice": correct_choice,
        "response": correct_answer
    })

  return Dataset.from_list(processed_data)

In [ ]:
# Create place to store llama 405B outputs
dataset = clean_data(data, labels)
default_values = ["None"] * len(dataset)
siqa_kd_train = dataset.add_column("405B_answer", default_values)
siqa_kd_train = siqa_kd_train.add_column("405B_predictions", default_values)
siqa_kd_train = siqa_kd_train.add_column("405B_logprobs", default_values)

In [ ]:
default_values = ["None"] * len(tests)
tests_405 = tests.add_column("405B_answer", default_values)

In [ ]:
print(siqa_kd_train[0])

{'prompt': 'Context: Cameron decided to have a barbecue and gathered her friends together.\nQuestion: How would Others feel as a result?\nOptions:\n  A: like attending\n  B: like staying home\n  C: a good friend to have\nThe correct answer is:\n', 'choice': 'A', 'response': 'like attending', '405B_answer': 'None', '405B_predictions': 'None', '405B_logprobs': 'None'}


In [24]:
import random
from tqdm import tqdm
import re

In [ ]:
# Run this for random sampling
siqa_kd_sample = siqa_kd_train.shuffle(seed=42).select(range(6000))

# **LLaMA 405B API Inference to get Logits and Top_k**

In [ ]:
import os
import openai

client = openai.OpenAI(
    api_key=###,
    base_url="https://api.hyperbolic.xyz/v1",
    )

CONTEXT = f"Given the context of the situation, you must choose one of the options to answer the question. No reasoning is needed."

def get_405B_answer(input, context=CONTEXT):
  response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct",
    messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": input["prompt"]},
    ],
    temperature=0.8,
    max_tokens=20,
    logprobs=True,
    top_logprobs=5,
    )

  return response

In [ ]:
def parse_response(row):
    response = get_405B_answer(row, context=CONTEXT)
    choice = response.choices[0]
    answer_text = choice.message.content
    # logprobs_list = choice.logprobs.content  # List of token logprobs per token

    # # Combine extraction of tokens and logprobs in one pass
    # combined = [
    #     (
    #         [tp.token for tp in token_item.top_logprobs],
    #         [tp.logprob for tp in token_item.top_logprobs]
    #     )
    #     for token_item in logprobs_list
    # ]

    # # Unzip the combined list into separate lists; handle empty case
    # if combined:
    #     token_predictions, token_logprobs = map(list, zip(*combined))
    # else:
    #     token_predictions, token_logprobs = [], []

    row["405B_answer"] = answer_text
    # row["405B_predictions"] = token_predictions
    # row["405B_logprobs"] = token_logprobs

    return row

In [ ]:
tests_405_answers = tests_405.map(parse_response)
tests_405_answers.save_to_disk("./tests_405_answers")

Map:   0%|          | 0/1954 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1954 [00:00<?, ? examples/s]

**Create new dataset with LLaMA 405B responses (run once)**

In [ ]:
# Adds entries to the two columns
new_dataset = siqa_kd_sample.map(parse_response)

new_dataset.save_to_disk("./siqa_kd_sample")

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6000 [00:00<?, ? examples/s]

# **Knowledge Distillation**

In [5]:
from datasets import load_from_disk

tests_405_answers = load_from_disk("./tests_405_answers")

**Load the saved dataset**

In [ ]:
from datasets import load_from_disk

kd_data = load_from_disk("./siqa_kd_sample")

In [ ]:
print(kd_data)

Dataset({
    features: ['prompt', 'choice', 'response', '405B_answer', '405B_predictions', '405B_logprobs'],
    num_rows: 6000
})


In [ ]:
print(kd_data[5])

{'prompt': 'Context: Cameron took sides and was disappointed to see neither accepted him.\nQuestion: What will happen to Cameron?\nOptions:\n  A: be disappointed next\n  B: be happy next\n  C: be welcomed next\nThe correct answer is:\n', 'choice': 'A', 'response': 'be disappointed next', '405B_answer': 'A: be disappointed next', '405B_predictions': [['A', 'The', 'C', 'Since', 'B'], [':', '', '.', ',', '<|eom_id|>'], [' be', '', ' Be', ' Cameron', ' He'], [' disappointed', ' disappointment', ' disappointing', ' diss', ' disappeared'], [' next', '', ' now', '.', '\tnext'], ['', '.', ',', '<|eom_id|>', ' \n\n']], '405B_logprobs': [[-0.0008526980527676642, -7.805732727050781, -8.274024963378906, -9.054512023925781, -9.678901672363281], [-0.16619941592216492, -1.8832709789276123, -6.878393650054932, -15.073515892028809, -15.775954246520996], [-1.1920928244535389e-07, -16.702438354492188, -18.26341438293457, -21.22926902770996, -22.16585350036621], [-1.1920928244535389e-07, -16.7024383544921

In [ ]:
import random
from tqdm import tqdm
import re

In [ ]:
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

**Load Student Model with LoRA**

In [9]:
# LLaMA 3.2 3B
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", device_map="auto")

# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = model.config.eos_token_id

special_tokens_dict = {"pad_token": "<pad>"}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)
if num_added_tokens > 0:
  model.resize_token_embeddings(len(tokenizer))

# tokenizer.pad_token = '<|finetune_right_pad_id|>'

# EOS_TOKEN = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16,                       # LoRA rank
    lora_alpha=32,            # LoRA scaling
    lora_dropout=0.05,        # dropout for LoRA
    bias="none",              # LoRA doesn't add bias by default
    task_type=TaskType.CAUSAL_LM,  # for a causal language model
    # target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

# Inject LoRA adapters into the original model
lora_model = get_peft_model(model, lora_config)

In [ ]:
lora_model.print_trainable_parameters()

trainable params: 4,587,520 || all params: 3,217,340,416 || trainable%: 0.1426


# **Black box KD**

In [ ]:
def tokenize_kd(example):
    prompt = example["prompt"]
    full_text = prompt + example["405B_answer"]

    prompt_enc = tokenizer(prompt, add_special_tokens=True)
    full_enc = tokenizer(full_text, truncation=True, padding="max_length", add_special_tokens=True, max_length=200, return_tensors="pt")

    # Copy labels
    labels = full_enc["input_ids"].clone()
    labels = labels[0].tolist()

    # Number of prompt tokens
    num_prompt_tokens = len(prompt_enc["input_ids"])

    for i in range(len(labels)):
      if i < num_prompt_tokens:
        labels[i] = -100
      elif labels[i] == tokenizer.pad_token_id:
        labels[i] = -100
        full_enc["attention_mask"][0][i] = 0

    # Convert the tensors back to lists so the dataset mapping function works as expected
    full_enc["input_ids"] = full_enc["input_ids"][0].tolist()
    full_enc["attention_mask"] = full_enc["attention_mask"][0].tolist()
    full_enc["labels"] = labels

    return full_enc

In [ ]:
print(kd_data[0])

{'prompt': 'Context: Alex was in a car accident and they eased their pain with rest.\nQuestion: What will Alex want to do next?\nOptions:\n  A: get into bed\n  B: Go to school\n  C: Call the doctor\nThe correct answer is:\n', 'choice': 'A', 'response': 'get into bed', '405B_answer': 'C', '405B_predictions': [['C', 'A', 'The', 'Since', 'B'], ['', ':', '.', ' Call', ')']], '405B_logprobs': [[-0.0007830176036804914, -7.961760520935059, -8.430052757263184, -8.586150169372559, -11.239808082580566], [-3.617743730545044, -0.02749757096171379, -8.30066967010498, -10.486035346984863, -12.983595848083496]]}


In [ ]:
kd_train = kd_data.map(tokenize_kd)
kd_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
print(kd_train[0])

{'input_ids': tensor([128000,   2014,     25,   8683,    574,    304,    264,   1841,  11677,
           323,    814,  95982,    872,   6784,    449,   2800,    627,  14924,
            25,   3639,    690,   8683,   1390,    311,    656,   1828,   5380,
          3883,    512,    220,    362,     25,    636,   1139,   4950,    198,
           220,    426,     25,   6122,    311,   2978,    198,    220,    356,
            25,   7290,    279,  10896,    198,    791,   4495,   4320,    374,
           512,     34, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
        128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
        128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
        128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
        128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
        128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256, 128256,
        128256, 128256, 12

In [ ]:
output_dir = "./siqa_bbkd"

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=5,  # start small to avoid OOM
    gradient_accumulation_steps=16,  # effective batch size = 4
    num_train_epochs=5,            # set higher if you have the compute
    learning_rate=2e-4,            # LoRA can often use a slightly higher LR
    logging_steps=5,
    save_steps=100,
    fp16=True,                     # if on GPU that supports half precision
    gradient_checkpointing=True,
    logging_dir="./logs",
    optim="adamw_torch",
    weight_decay=0.01, # do 0.005 or 0.001?
    warmup_steps=50
)

trainer = Trainer(
    model=lora_model,      # This is the lora model that should already be set up
    args=training_args,
    train_dataset=kd_train,
    # eval_dataset=eval_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.save_model(output_dir)

Step,Training Loss
5,0.927100
10,0.676100
15,0.506000
20,0.460400
25,0.404200
30,0.498400
35,0.443000
40,0.487800
45,0.292100
50,0.341600


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

# **White Box KD**

In [ ]:
# This function returns the token_id of a given token and handles any errors

def convert_token(token, tokenizer):
    token_stripped = token.strip()
    if not token_stripped:
        return tokenizer.unk_token_id if tokenizer.unk_token_id is not None else tokenizer.eos_token_id
    token_id = tokenizer.convert_tokens_to_ids(token_stripped)
    if token_id is None or token_id < 0:
        return tokenizer.unk_token_id if tokenizer.unk_token_id is not None else tokenizer.eos_token_id
    return token_id

In [ ]:
print(kd_data[0]["405B_logprobs"])

[[-0.0007830176036804914, -7.961760520935059, -8.430052757263184, -8.586150169372559, -11.239808082580566], [-3.617743730545044, -0.02749757096171379, -8.30066967010498, -10.486035346984863, -12.983595848083496]]


In [ ]:
# This function tokenizes the prompts as needed and builds the input dictionary for training
def tokenize_soft(example):
    prompt = example["prompt"]
    full_text = prompt + example["choice"] + ": " + example["response"]

    prompt_enc = tokenizer(prompt, add_special_tokens=True)
    full_enc = tokenizer(full_text, truncation=True, padding="max_length", add_special_tokens=True, max_length=200, return_tensors="pt")

    # Copy labels
    labels = full_enc["input_ids"].clone()
    labels = labels[0].tolist()

    # Number of prompt tokens
    num_prompt_tokens = len(prompt_enc["input_ids"])

    for i in range(len(labels)):
      if i < num_prompt_tokens:
        labels[i] = -100
      elif labels[i] == tokenizer.pad_token_id:
        labels[i] = -100
        full_enc["attention_mask"][0][i] = 0

    # EVERYTHING ABOVE SHOULD BE YOUR NORMAL TOKENIZER FUNCTION; adapt for different datasets
    # The following adds two more columns (or two more keys to the dictionary)

    # Convert each of the tokens in the list of lists to token_ids
    teacher_preds_ids = [
    [convert_token(token, tokenizer) for token in token_list]
    for token_list in example["405B_predictions"]
]

    # may or may not need this (make sure all logprobs are floats)
    teacher_logprobs = [
        [float(obj) for obj in list]
        for list in example["405B_logprobs"]
        ]

    # Convert the tensors back to lists so the dataset mapping function works as expected
    full_enc["input_ids"] = full_enc["input_ids"][0].tolist()
    full_enc["attention_mask"] = full_enc["attention_mask"][0].tolist()
    full_enc["labels"] = labels
    full_enc["teacher_predictions"] = teacher_preds_ids
    full_enc["teacher_logprobs"] = teacher_logprobs

    return full_enc

**Note: It is assumed that "kd_data" has been previously loaded as a dataset, and contains the columns used in the above function by example["key"]**

In [ ]:
distill_train = kd_data.map(tokenize_soft)
# This builds dictionary of tensors
distill_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "teacher_predictions", "teacher_logprobs"])

In [ ]:
print(kd_data[0]["405B_predictions"])

[['C', 'A', 'The', 'Since', 'B'], ['', ':', '.', ' Call', ')']]


In [ ]:
print(distill_train[0]["teacher_predictions"])

[[34, 32, 791, 12834, 33], [128001, 25, 13, 7368, 8]]


In [ ]:
print(distill_train[0]["teacher_logprobs"])

tensor([[-7.8302e-04, -7.9618e+00, -8.4301e+00, -8.5862e+00, -1.1240e+01],
        [-3.6177e+00, -2.7498e-02, -8.3007e+00, -1.0486e+01, -1.2984e+01]])


In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer

# def recursive_convert_to_float(obj):
#     if isinstance(obj, list):
#         return [recursive_convert_to_float(x) for x in obj]
#     return float(obj)

# Helper function to compute teacher probability distribution from teacher_logprobs.
def compute_teacher_distribution(teacher_logprobs):
    # If teacher_logprobs is a list, convert it to a tensor.
    # if isinstance(teacher_logprobs, list):
    #     teacher_logprobs = recursive_convert_to_float(teacher_logprobs)
    #     teacher_logprobs = torch.tensor(teacher_logprobs, dtype=torch.float)
    # Convert negative log probs to probs (p = exp(-nlogp)), then normalize over the top-k tokens. (shape: [B, S, 5])
    teacher_probs = torch.exp(-teacher_logprobs)
    teacher_probs = teacher_probs / teacher_probs.sum(dim=-1, keepdim=True)
    return teacher_probs

class DistillationTrainer(Trainer):
    def __init__(self, *args, alpha=0.5, beta=0.5, **kwargs):
        """
        alpha: weight for the distillation (KL) loss.
        beta: weight for the ground truth (CE) loss.
        """
        super().__init__(*args, **kwargs)
        self.alpha = alpha  # weight for forward vs. reverse KL loss
        self.beta = beta    # weight for the ground-truth CE loss vs KL

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Extract teacher predictions and logprobs from the inputs and remove them from the dict.
        teacher_preds = inputs.pop("teacher_predictions") # expected shape: [B, seq_length, top_k]
        teacher_logprobs = inputs.pop("teacher_logprobs")   # expected shape: [B, seq_length, top_k]

        # model_inputs = {key: inputs[key] for key in ["input_ids", "attention_mask", "labels"] if key in inputs}
        outputs = model(**inputs)
        logits = outputs.logits  # shape: [B, seq_length, vocab_size]

        # 1. Compute cross-entropy loss with the ground truth.
        labels = inputs.get("labels")
        loss_ce = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=-100
        )

        # 2. Compute the distillation loss on the teacher’s token predictions.
        # Convert teacher_logprobs into a probability distribution.
        teacher_probs = compute_teacher_distribution(teacher_logprobs)  # shape: [B, seq_length, top_k]

        # Gather the student logits for the teacher's top-k tokens.
        # teacher_preds should contain the token indices corresponding to the top-k teacher predictions.
        student_logits_top = torch.gather(logits, dim=-1, index=teacher_preds)
        student_probs_top = F.softmax(student_logits_top, dim=-1)

        # Compute forward KL: KL(teacher || student) = Σ teacher_probs * (log(teacher_probs) - log(student_probs_top))
        forward_kl = (teacher_probs * (torch.log(teacher_probs + 1e-8) - torch.log(student_probs_top + 1e-8))).sum(dim=-1).mean()

        # Compute reverse KL: KL(student || teacher) = Σ student_probs_top * (log(student_probs_top) - log(teacher_probs))
        reverse_kl = (student_probs_top * (torch.log(student_probs_top + 1e-8) - torch.log(teacher_probs + 1e-8))).sum(dim=-1).mean()

        # Combine the two KL terms.
        distill_loss = self.alpha * forward_kl + (1 - self.alpha) * reverse_kl

        # Combine the losses.
        loss = (1 - self.beta) * distill_loss + self.beta * loss_ce
        # print("Loss grad_fn:", loss.grad_fn)

        if return_outputs:
            return loss, outputs
        return loss


In [ ]:
alpha = 0  # Weight for forward KL; (1-alpha) for reverse KL loss
beta = 1  # Weight for the ground truth (CE) loss; (1-beta) for total KL loss

output_dir = output_dir = f"./siqa_alpha={alpha}_beta={beta}" # saves in a different dir for different params

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Set up training arguments.
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1, # Increasing batch size might cuase problems
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    logging_steps=5, # adjust as needed
    save_steps=500, # adjust as needed
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=False,                     # needs to be turned off!
    gradient_checkpointing=True,
    logging_dir="./logs",
    # optim="adamw_torch",
    warmup_steps=75,  #This should be around 5-10% of total steps
    remove_unused_columns=False,  # to include additional columns
    report_to=["wandb"]
)

# Instantiate the custom distillation trainer.
trainer = DistillationTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=distill_train,  # prepared dataset here
    # tokenizer=tokenizer,
    # data_collator=data_collator,
    alpha=alpha,
    beta=beta
)

# Start training.
trainer.train()
trainer.save_model(output_dir)

Step,Training Loss
5,0.002300
10,0.000900


KeyboardInterrupt: 

# **Fine Tuning Student**

In [ ]:
def tokenize_fn(example):
    prompt = example["prompt"]
    full_text = prompt + example["choice"] + ": " + example["response"]

    prompt_enc = tokenizer(prompt, add_special_tokens=True)
    full_enc = tokenizer(full_text, truncation=True, padding="max_length", add_special_tokens=True, max_length=200, return_tensors="pt")

    # Copy labels
    labels = full_enc["input_ids"].clone()
    labels = labels[0].tolist()

    # Number of prompt tokens
    num_prompt_tokens = len(prompt_enc["input_ids"])

    for i in range(len(labels)):
      if i < num_prompt_tokens:
        labels[i] = -100
      elif labels[i] == tokenizer.pad_token_id:
        labels[i] = -100
        full_enc["attention_mask"][0][i] = 0

    # Convert the tensors back to lists so the dataset mapping function works as expected
    full_enc["input_ids"] = full_enc["input_ids"][0].tolist()
    full_enc["attention_mask"] = full_enc["attention_mask"][0].tolist()
    full_enc["labels"] = labels

    return full_enc

In [ ]:
train_dataset = kd_data.map(tokenize_fn)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
print(tokenizer.decode(train_dataset[45]["input_ids"], skip_special_tokens=True))

Context: Casey went over to Jan's house and got the trash to throw away for her.
Question: What does Casey need to do before this?
Options:
  A: put on shoes
  B: thank Casey for throwing away the trash
  C: put away her trash
The correct answer is:
A: put on shoes


In [ ]:
# set output dir
output_dir = "./siqa_ft6000"

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=5,  # start small to avoid OOM
    gradient_accumulation_steps=16,  # effective batch size = 5 * 16
    num_train_epochs=5,            # set higher if you have the compute
    learning_rate=2e-4,            # LoRA can often use a slightly higher LR
    logging_steps=5,
    save_steps=50,
    fp16=True,                     # if on GPU that supports half precision
    gradient_checkpointing=True,
    logging_dir="./logs",
    optim="adamw_torch",
    weight_decay=0.01, # do 0.005 or 0.001?
    warmup_steps=40
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.save_model(output_dir)

Step,Training Loss
5,23.342900
10,16.796400
15,7.731100
20,1.958800
25,0.447600
30,0.241800
35,0.227600
40,0.203600
45,0.197700
50,0.198500


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

# **EVAL**

In [16]:
test_data, test_answers = load_data(test_file, test_labels)

In [ ]:
print(test_data[0])

{'context': "Tracy didn't go home that evening and resisted Riley's attacks.", 'question': 'What does Tracy need to do before this?', 'answerA': 'make a new plan', 'answerB': 'Go home and see Riley', 'answerC': 'Find somewhere to go'}


In [19]:
tests = clean_data(test_data, test_answers)

In [ ]:
print(tests)

Dataset({
    features: ['prompt', 'choice', 'response'],
    num_rows: 1954
})


In [6]:
def strip_response(output):
  pattern = r"The correct answer is:\n(.*?[.,!?\-()\n])"

  match = re.search(pattern, output)
  if match:
    sentence = match.group(1)
    # Remove trailing punctuation if present
    sentence = sentence.rstrip(".,!?\-()")
    return sentence.strip()
  else:
    return ""

In [7]:
def get_correct_answer(example):
  correct_answer = example["choice"] + ": " + example["response"]
  return correct_answer

In [ ]:
# eval_path = f"./siqa_alpha={alpha}_beta={beta}"
eval_path = "./siqa_ft6000"

In [ ]:
from operator import ge
from transformers import pipeline
from peft import PeftModel

# Replace "./fine_tuned_model" with the path to your fine-tuned model
adapted_model = PeftModel.from_pretrained(model, eval_path)
merged_model = adapted_model.merge_and_unload()
gen_pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
prompt = tests[7]["prompt"]
print(prompt)
outputs = gen_pipe(prompt, max_new_tokens=20, top_p=0.95, temperature=0.8)
print(strip_response(outputs[0]["generated_text"]))

Context: Kendall got a new sports car and could not wait to show friends.
Question: What will Kendall want to do next?
Options:
  A: drive that sports car
  B: show off his new sports car
  C: clean and wax her legs
The correct answer is:

B: show off his new sports car


In [22]:
import pandas as pd
from tqdm import tqdm

def eval(gen_pipe, tests):
  answers = []
  scores = []

  for test in tqdm(tests):
    prompt = test["prompt"]
    outputs = gen_pipe(prompt, max_new_tokens=20, top_p=0.95, temperature=0.8)
    answer = strip_response(outputs[0]["generated_text"])
    answers.append(answer)
    if answer.lower() == get_correct_answer(test).lower():
      scores.append(1)
    else:
      scores.append(0)

  return answers, scores

A: make a new plan


In [8]:
answers = []
scores = []

for test in tests_405_answers:
  answer = test["405B_answer"]
  answers.append(answer)
  if answer.lower() == get_correct_answer(test).lower():
    scores.append(1)
  else:
    scores.append(0)

# compute accuracy
total = len(scores)
correct = sum(scores)
accuracy = correct / total
model_description = "405B-inference"

if model_description != "":
  print(model_description)

print("Number of examples processed: " + str(total))
print("Number of correct predictions: " + str(correct))
print("Accuracy: " + str(accuracy))

405B-inference
Number of examples processed: 1954
Number of correct predictions: 1281
Accuracy: 0.6555783009211873


In [11]:
from operator import ge
from transformers import pipeline

new_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [12]:
eval_path = "./siqa_3b_noft"

In [25]:
# Create a DataFrame with each list as a column
ans, sco = eval(new_pipe, tests)

df = pd.DataFrame({
    "answers": ans,
    "correctness": sco
})

save_path = eval_path + "/eval_results.csv"

# Save the DataFrame to a new CSV file without the index column
df.to_csv(save_path, index=False)

100%|██████████| 1954/1954 [13:29<00:00,  2.41it/s]


OSError: Cannot save file into a non-existent directory: 'siqa_3b_noft'

In [26]:
# compute accuracy
total = len(sco)
correct = sum(sco)
accuracy = correct / total
model_description = eval_path

if model_description != "":
  print(model_description)

print("Number of examples processed: " + str(total))
print("Number of correct predictions: " + str(correct))
print("Accuracy: " + str(accuracy))

./siqa_3b_noft
Number of examples processed: 1954
Number of correct predictions: 356
Accuracy: 0.18219037871033777


**Load Lora Model**

In [ ]:
eval_path = output_dir

In [ ]:
print(eval_path)

./gsm8k_ft800


In [ ]:
from operator import ge
from transformers import pipeline
from peft import PeftModel

# Replace "./fine_tuned_model" with the path to your fine-tuned model
adapted_model = PeftModel.from_pretrained(model, eval_path)
merged_model = adapted_model.merge_and_unload()
gen_pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer)

Device set to use cuda:0


In [27]:
def clean_answer(model_pred, n_shot):
    model_pred = model_pred.lower()
    preds = model_pred.split(ANSWER_TRIGGER.lower())
    answer_flag = True if len(preds) > n_shot + 1 else False
    if answer_flag:
        # Pick first answer with flag
        pred = preds[n_shot + 1]
    else:
        # Pick last number without flag
        pred = preds[-1]

    pred = pred.replace(",", "")
    pred = [s for s in re.findall(r"-?\d+\.?\d*", pred)]

    if len(pred) == 0:
        return INVALID_ANS

    if answer_flag:
        # choose the first element in list
        pred = pred[0]
    else:
        # choose the last element in list
        pred = pred[-1]

    # (For arithmetic tasks) if a word ends with period, it will be omitted ...
    if pred[-1] == ".":
        pred = pred[:-1]

    return pred

In [28]:
def extract_answer_from_output(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS

In [ ]:
# example test
question = gsm8k_test[7]["question"]
demo = create_demo_text(2, COT_FLAG)
prompt = demo + "Q: " + question + "\n" + "A: "
outputs = gen_pipe(prompt, max_new_tokens=256, top_p=0.95, temperature=0.8)
print(outputs[0]["generated_text"])

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is:  9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is:  29.

Q: Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?
A: 200 GB is 200,000,000 MB. 2 GB / minute is 2,000,000,000 B / minute. 40% of the way through the download is 80,000,000 B. 80,000,000 B + 

**EVAL**

In [29]:
from datasets import load_dataset

gsm8k_train = load_dataset("openai/gsm8k", "main", split="train")
gsm8k_test = load_dataset("openai/gsm8k", "main", split="test")

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [30]:
test_dataset = gsm8k_test.shuffle(seed=42).select(range(200))

In [ ]:
input_text = build_prompt(test_dataset[0], 2, COT_FLAG)
generate_kwargs = dict(max_new_tokens=256, top_p=0.95, temperature=0.8)
merged_model.eval()
model_completion = generate(merged_model, tokenizer, input_text, generate_kwargs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(model_completion)

In [31]:
def generate(model, tokenizer, input_text, generate_kwargs):
    input_text = tokenizer(
        input_text,
        padding=False,
        add_special_tokens=True,
        return_tensors="pt",
    )
    input_ids = input_text.input_ids.cuda()
    attention_mask = input_text.attention_mask.cuda()

    output_ids = model.generate(
        input_ids=input_ids, attention_mask=attention_mask, **generate_kwargs
    )
    response = []
    for i in range(output_ids.shape[0]):
        response.append(
            tokenizer.decode(
                output_ids[i][input_ids.shape[1] :],
                skip_special_tokens=True,
                ignore_tokenization_space=True,
            )
        )

    if len(response) > 1:
        return response
    return response[0]

In [37]:
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"

N_SHOT = 8
COT_FLAG = True
DEBUG = False
ANSWER_TRIGGER = "The answer is: "

In [41]:
from tqdm import tqdm

answers = []
scores = []

for sample in tqdm(test_dataset):
  input_text = build_prompt(sample, 2, True)
  generate_kwargs = dict(max_new_tokens=256, top_p=0.95, temperature=0.8)
  model_completion = generate(model, tokenizer, input_text, generate_kwargs)
  model_answer = clean_answer(model_completion, 2)
  answers.append(model_answer)

  true_answer = extract_answer_from_output(sample["answer"])

  if (model_answer == true_answer):
    scores.append(1)
  else:
    scores.append(0)

100%|██████████| 200/200 [25:38<00:00,  7.69s/it]


In [ ]:
import pandas as pd

# Create a DataFrame with each list as a column
df = pd.DataFrame({
    "answers": answers,
    "correctness": scores
})

save_path = eval_path + "/eval_results.csv"

# Save the DataFrame to a new CSV file without the index column
df.to_csv(save_path, index=False)

In [42]:
print(
    f"Model type: gsm8k 3b raw, "
    f"Num of total question: {len(scores)}, "
    f"Correct num: {sum(scores)}, "
    f"Accuracy: {float(sum(scores))/len(scores)}."
)

Model type: gsm8k 3b raw, Num of total question: 200, Correct num: 8, Accuracy: 0.04.


In [34]:
def build_prompt(input, n_shot, cot_flag):
    demo = create_demo_text(n_shot, cot_flag)
    input_text_prompt = demo + "Q: " + input['question'] + "\n" + "A: "
    return input_text_prompt

In [33]:
# Demo Examples for CoT
def create_demo_text(n_shot=8, cot_flag=True):
    question, chain, answer = [], [], []
    question.append(
        "There are 15 trees in the grove. "
        "Grove workers will plant trees in the grove today. "
        "After they are done, there will be 21 trees. "
        "How many trees did the grove workers plant today?"
    )
    chain.append(
        "There are 15 trees originally. "
        "Then there were 21 trees after some more were planted. "
        "So there must have been 21 - 15 = 6."
    )
    answer.append("6")

    question.append(
        "If there are 3 cars in the parking lot and 2 more cars arrive, "
        "how many cars are in the parking lot?"
    )
    chain.append("There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.")
    answer.append("5")

    question.append(
        "Leah had 32 chocolates and her sister had 42. If they ate 35, "
        "how many pieces do they have left in total?"
    )
    chain.append(
        "Originally, Leah had 32 chocolates. "
        "Her sister had 42. So in total they had 32 + 42 = 74. "
        "After eating 35, they had 74 - 35 = 39."
    )
    answer.append("39")

    question.append(
        "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason "
        "has 12 lollipops. How many lollipops did Jason give to Denny?"
    )
    chain.append(
        "Jason started with 20 lollipops. Then he had 12 after giving some "
        "to Denny. So he gave Denny 20 - 12 = 8."
    )
    answer.append("8")

    question.append(
        "Shawn has five toys. For Christmas, he got two toys each from his "
        "mom and dad. How many toys does he have now?"
    )
    chain.append(
        "Shawn started with 5 toys. If he got 2 toys each from his mom and "
        "dad, then that is 4 more toys. 5 + 4 = 9."
    )
    answer.append("9")

    question.append(
        "There were nine computers in the server room. Five more computers "
        "were installed each day, from monday to thursday. "
        "How many computers are now in the server room?"
    )
    chain.append(
        "There were originally 9 computers. For each of 4 days, 5 more "
        "computers were added. So 5 * 4 = 20 computers were added. "
        "9 + 20 is 29."
    )
    answer.append("29")

    question.append(
        "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On "
        "wednesday, he lost 2 more. "
        "How many golf balls did he have at the end of wednesday?"
    )
    chain.append(
        "Michael started with 58 golf balls. After losing 23 on tuesday, "
        "he had 58 - 23 = 35. After losing 2 more, "
        "he had 35 - 2 = 33 golf balls."
    )
    answer.append("33")

    question.append(
        "Olivia has $23. She bought five bagels for $3 each. "
        "How much money does she have left?"
    )
    chain.append(
        "Olivia had 23 dollars. "
        "5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. "
        "So she has 23 - 15 dollars left. 23 - 15 is 8."
    )
    answer.append("8")

    # randomize order of the examples ...
    index_list = list(range(len(question)))
    random.shuffle(index_list)

    # Concatenate demonstration examples ...
    demo_text = ""
    for i in index_list[:n_shot]:
        if cot_flag:
            demo_text += (
                "Q: "
                + question[i]
                + "\nA: "
                + chain[i]
                + " "
                + ANSWER_TRIGGER
                + " "
                + answer[i]
                + ".\n\n"
            )
        else:
            demo_text += (
                "Question: "
                + question[i]
                + "\nAnswer: "
                + ANSWER_TRIGGER
                + " "
                + answer[i]
                + ".\n\n"
            )
    return demo_text